In [1]:


class MonthTimeValidation(object):
    def __init__(self, month_to_test_set=2, time_col='timestamp'):
        self.month_to_test_set = month_to_test_set
        self.time_col = time_col
        
    def split(self, df):
        split_col = df[self.time_col].dt.month
        split_col = split_col.reset_index(drop=True)
        
        for max_month in range(1,13-self.month_to_test_set):
            train_idx = split_col[split_col <= max_month].index.tolist()
            test_idx = split_col[(split_col > max_month) & (split_col <= max_month+self.month_to_test_set)].index.tolist()
            yield train_idx, test_idx
            
import numpy as np


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


from sklearn.metrics import mean_squared_error

def LRMSE(y_true, y_pred):
    return (mean_squared_error(y_true,y_pred))**(1/2)



In [2]:


class NaiveMeanModel(object):
    def __init__(self, values_to_count_mean, target_variable_name, value_to_fillna=0):
        self.values_to_count_mean = values_to_count_mean
        self.target_variable_name = target_variable_name
        self.value_to_fillna = value_to_fillna
        
        self.counted_stats = None 
        
    def fit(self, X, y=None):
        if len(set(self.values_to_count_mean) & set(X.columns)) < len(self.values_to_count_mean):
            raise ValueError('Columns to count stats not in df')
            
        self.counted_stats = X.groupby(self.values_to_count_mean)[self.target_variable_name].mean().reset_index()
        
    def predict(self, X):
        if self.target_variable_name in X.columns:
            prediction =  X.merge(self.counted_stats, on=self.values_to_count_mean, how='left')[self.target_variable_name+'_y']
        else:
            prediction =  X.merge(self.counted_stats, on=self.values_to_count_mean, how='left')[self.target_variable_name]
            
        print(str(prediction.isna().sum()) + ' Nan detected')
        return prediction.fillna(self.value_to_fillna).reset_index(drop=True)



In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

class MyRegressor(object):
    def __init__(self, ml_params, nmm_params, tgt_variable='meter_reading'):
        self.ml = [RandomForestRegressor(**ml_params) for i in range(4)]
        self.naive_mean_model = NaiveMeanModel(**nmm_params)
        self.tgt_variable = tgt_variable
        
        self.mean = None
        self.std = None
        
    def fit(self, X):
        self.naive_mean_model.fit(X)
        X['stat'] = self.naive_mean_model.predict(X)
        
        for i in range(4):
            self.ml[i].fit(X[X['meter']==i].drop(columns=['timestamp',self.tgt_variable,'meter']), X.loc[X['meter']==i,self.tgt_variable])
        
        return self
    
    def predict(self, X):
        X['stat'] = self.naive_mean_model.predict(X)
        cols_to_drop = list({'row_id', 'timestamp', self.tgt_variable} & set(X.columns))
        
        X['prediction'] = 0
        for i in range(4):
            X.loc[X['meter']==i, 'prediction'] = self.ml[i].predict(X[X['meter']==i].drop(columns=cols_to_drop+['meter','prediction']))
        
        return X['prediction']


# Main

In [4]:
import pandas as pd
import numpy as np
import gc

from os import path

from matplotlib import pyplot as plt
%matplotlib inline

# Read data

In [5]:
data_path = '/kaggle/input/ashrae-energy-prediction/'

In [6]:
train = reduce_mem_usage(pd.read_csv(path.join(data_path,'train.csv')))
test = reduce_mem_usage(pd.read_csv(path.join(data_path,'test.csv')))

building_metadata = reduce_mem_usage(pd.read_csv(path.join(data_path,'building_metadata.csv')))
weather = reduce_mem_usage(pd.read_csv('/kaggle/input/ashrae/fixed_weather_df.csv')).drop(columns='Unnamed: 0')

Mem. usage decreased to 289.19 Mb (53.1% reduction)
Mem. usage decreased to 596.49 Mb (53.1% reduction)
Mem. usage decreased to  0.03 Mb (60.3% reduction)
Mem. usage decreased to 10.74 Mb (66.2% reduction)


# Prepare Data

In [7]:
WEATHER_FETURES_WITH_NANS = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']

In [8]:
def prepare_data(df, build_metadata, weather_metadata, is_test=False):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp')
    
    if not is_test:
        primary_use_dict = {el:i for i, el in enumerate(build_metadata['primary_use'].unique())}
        build_metadata['primary_use'] = build_metadata['primary_use'].map(primary_use_dict)
        
    if not is_test:
        weather_metadata['timestamp'] = pd.to_datetime(weather_metadata['timestamp'])
        weather_metadata = weather_metadata.sort_values('timestamp')
        
        for f in WEATHER_FETURES_WITH_NANS:
            weather_metadata[f] = weather_metadata[f].fillna(weather_metadata[f].median())
    
    df = df.merge(build_metadata, on='building_id', how='left')
    df = df.merge(weather_metadata, on=['site_id','timestamp'], how='left')
    
    df['day_of_week'] = df['timestamp'].dt.weekday
    df['hour'] = df['timestamp'].dt.hour
    
    df['year_built'] = df['year_built'].fillna(df['year_built'].median())
    df['floor_count'] = df['floor_count'].fillna(df['floor_count'].median())
    for f in WEATHER_FETURES_WITH_NANS:
            df[f] = df[f].fillna(df[f].median())
            
    if not is_test:
        df['meter_reading'] = np.log1p(df['meter_reading'])
    
    return df

In [9]:
train = prepare_data(train, building_metadata, weather)
test = prepare_data(test, building_metadata, weather, is_test=True)

In [10]:
del building_metadata
del weather
gc.collect()

50

# Random Forest Regressor

## Validation

In [11]:
def time_val(data, model, metric=LRMSE, target_var_name='meter_reading'):
    time_validation_split = MonthTimeValidation()
    
    results = []
    for train_idx, test_idx in time_validation_split.split(data):
        model.fit(data.iloc[train_idx].reset_index(drop=True))
        pred = model.predict(data.iloc[test_idx].reset_index(drop=True))
        itter_metric = metric(data.iloc[test_idx][target_var_name], pred)
        
        print('Itter metric: '+str(itter_metric))
        results.append(itter_metric)
        
        gc.collect()
        
    return results

In [12]:
rf_model = MyRegressor(ml_params={'n_estimators':50, 'criterion':'mse', 'max_depth':17, 'max_features':'sqrt', 'n_jobs':4}, 
                       nmm_params={'values_to_count_mean':['building_id','meter','day_of_week','hour'], 'target_variable_name':'meter_reading'})

In [13]:
rf_res = time_val(train, rf_model)
print('Result: {} +/- {}'.format(round(np.mean(rf_res),5), round(np.std(rf_res),5)))

0 Nan detected
32335 Nan detected
Itter metric: 1.1681758236305184
0 Nan detected
36001 Nan detected
Itter metric: 1.188940167142598
0 Nan detected
8174 Nan detected
Itter metric: 1.213690719279122
0 Nan detected
6474 Nan detected
Itter metric: 1.487707991236372
0 Nan detected
2744 Nan detected
Itter metric: 1.3847829887046823
0 Nan detected
1917 Nan detected
Itter metric: 1.0534195500860117
0 Nan detected
6568 Nan detected
Itter metric: 1.0031705684226095
0 Nan detected
4533 Nan detected
Itter metric: 1.297020475533684
0 Nan detected
1520 Nan detected
Itter metric: 1.4906479592693327
0 Nan detected
2276 Nan detected
Itter metric: 1.3820539439853434
Result: 1.26696 +/- 0.16145


## Predict

In [14]:
rf_model.fit(train)
test['meter_reading'] = rf_model.predict(test)

0 Nan detected
0 Nan detected


In [15]:
test['meter_reading'] = np.expm1(test['meter_reading'])

In [16]:
test.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,...,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,day_of_week,hour,stat,prediction,meter_reading
0,0,0,0,2017-01-01,0,0,7432,2008.0,3.0,15.601562,...,12.796875,0.0,1022.0,130.0,2.099609,6,0,3.347265,1.280242,2.597509
1,41614356,1393,0,2017-01-01,15,3,151900,1990.0,3.0,2.800781,...,0.000000,0.0,1008.0,210.0,4.101562,6,0,4.400648,4.342333,75.886744
2,41614355,1392,2,2017-01-01,15,3,76153,1963.0,3.0,2.800781,...,0.000000,0.0,1008.0,210.0,4.101562,6,0,7.538067,7.722743,2258.147224
3,41614354,1392,0,2017-01-01,15,3,76153,1963.0,3.0,2.800781,...,0.000000,0.0,1008.0,210.0,4.101562,6,0,4.167542,4.190253,65.039525
4,41614353,1391,2,2017-01-01,15,3,68693,1956.0,3.0,2.800781,...,0.000000,0.0,1008.0,210.0,4.101562,6,0,6.801698,7.077190,1183.634530


In [17]:
test[['row_id','meter_reading']].to_csv('naive_mean_predictor.csv', index=False)